In [4]:
import warnings
import logging


warnings.filterwarnings('ignore',category=FutureWarning)
warnings.simplefilter('ignore')

In [5]:
from utils import make_environments
from utils import pearl_utils
from configs import defaults
from utils.reward_functions import log_reward_function,cumulative_reward_function,sharpe_reward_function
from utils. utils import make_hidden_dims
import optuna
from optuna.samplers import TPESampler

from neuralforecast.core import NeuralForecast
from Pearl.pearl.utils.instantiations.environments.gym_environment import GymEnvironment
from Pearl.pearl.utils.functional_utils.train_and_eval.online_learning import \
    online_learning
import datetime

In [6]:
reward_functions=[log_reward_function,cumulative_reward_function,sharpe_reward_function]
train_env,test_env=make_environments.make_envs(reward_function=log_reward_function)


Seed set to 12
Seed set to 5


['data_forex/oanda-USDJPY-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 12336.19it/s]
1it [00:00,  9.37it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

2024-11-04 16:00:00 2024-12-04 16:00:00


In [7]:
sp=train_env.positions
train_env.action_space.n,train_env.name,train_env.observation_space.shape,

(np.int64(2), 'USDJPY_train', (24,))

In [8]:
make_hidden_dims(n_layers=3, n_units=64)

[64, 64, 64]

In [9]:
agent=pearl_utils.create_ddqn_model(

        observation_space_dim=train_env.observation_space.shape[0], 
        action_space_dim=train_env.action_space.n,
        hidden_dims=[64,64, 64], 
        training_rounds=20,
        learning_rate = 0.001,
        discount_factor = 0.99,
        batch_size = 128,
        target_update_freq = 10,
        soft_update_tau = 0.75,  # a value of 1 indicates no soft updates
        is_conservative = False,
        conservative_alpha = False,
        replay_buffer_size = 10_000,
        lstm=False)

In [10]:
test_env.observation_space.shape,train_env.action_space.n

((24,), np.int64(2))

In [11]:
env=GymEnvironment(train_env)

obs,action_space=env.reset()
agent.reset(   obs, action_space)

In [12]:
# done = False
# while not done:
#     action = agent.act(exploit=False)
#     action_result = env.step(action)
    
#     agent.observe(action_result)
#     loss=agent.learn()

#     done = action_result.done

In [13]:
info = online_learning(
        agent=agent,
        env=env,
        # number_of_episodes=10,
        number_of_steps=168,
        print_every_x_episodes=2,   # print returns after every 10 episdoes
        print_every_x_steps=1,   # print returns after every 10 episdoes
        learn_every_k_steps=20,   # print returns after every 10 episdoes
        learn_after_episode=False,
        record_period=169,   # instead of updating after every environment interaction, Q networks are updates at the end of each episode
        seed=0
    )

  0%|          | 0/168 [00:00<?, ?it/s]

In [20]:


def objective_function(trial):
 
    reward_id=trial.suggest_categorical('reward_function', [0,1,2])
    algo=trial.suggest_categorical('algorithm', ['dqn','ddqn'])    

    # reward_id=0
    
    reward_func=reward_functions[reward_id]
    train_env.reward_func=reward_func
    test_env.reward_func=reward_func
    
    observation_space_dim=train_env.observation_space.shape[0]
    action_space_dim=len(train_env.positions)
    n_layers=trial.suggest_int('n_layers', 1, 3)
    n_units=trial.suggest_categorical('n_units', [64,128,256,512])
    
    hidden_dims=make_hidden_dims(n_layers= n_layers, n_units=n_units)
    
    search_space={
                'observation_space_dim': observation_space_dim,
                'action_space_dim': action_space_dim,
                'hidden_dims': hidden_dims,
                'training_rounds': trial.suggest_int('training_rounds', 5, 30),
                'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-4),
                'discount_factor': trial.suggest_float('discount_factor', 0.8, 0.99), # gamma (greediness)
                'batch_size': trial.suggest_categorical('batch_size', [64, 128]),
                'target_update_freq': trial.suggest_categorical('target_update_freq', [1, 5, 10, 24]),
                'soft_update_tau': trial.suggest_float('soft_update_tau', 0.1, .99), 
                'is_conservative': trial.suggest_categorical('is_conservative', [True, False]),
                'lstm': trial.suggest_categorical('lstm', [True, False]),
                'conservative_alpha': trial.suggest_float('conservative_alpha', 0.5, 1.0),
                'replay_buffer_size': trial.suggest_categorical('replay_buffer_size', [1_000,10_000,50_000,100_000,200_000]),

                }

    learning_space={'learn_after_episode':trial.suggest_categorical('learn_after_episode', [True, False]),
                    'learning_steps':trial.suggest_int('learning_steps', 10, 89),
                    'n_epochs':trial.suggest_categorical('n_epochs',[100,500]),
                    }
    #
    # print('n_epochs',n_epochs)
    if algo=='dqn':
        agent=pearl_utils.create_dqn_model(**search_space)
    elif algo=='ddqn':
        agent=pearl_utils.create_ddqn_model(**search_space)

        
    agent=pearl_utils.train_pearl_model(agent,train_env,**learning_space)
    profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)
    objectives={'profit':profit,'n_trades':n_trades}

    print('profit',profit,'n_trades',n_trades)

    return profit,n_trades

In [21]:
model_name=defaults.model_name
model_name

study_name=f"{defaults.model_name}"
storage_name="sqlite:///PearlHPTuning.sqlite3"

In [22]:
from optuna import create_study,delete_study

In [23]:
# study=delete_study(study_name=study_name, 
#              storage=storage_name, )

In [24]:

study=create_study(study_name=study_name, 
             storage=storage_name, 
             load_if_exists=True,
             directions=['maximize','maximize'],
             sampler=TPESampler()
             )

[I 2024-12-04 15:53:06,449] A new study created in RDB with name: USDJPY


In [25]:
study.optimize(objective_function, n_trials=100)


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/84000 [00:00<?, ?it/s]

  0%|          | 0/500 [08:23<?, ?it/s]
Profit: 1150.5957618769667, Number of Trades: 18.4: 100%|██████████| 100/100 [00:35<00:00,  2.82it/s]             
[I 2024-12-04 16:02:05,680] Trial 0 finished with values: [1150.5957618769667, 18.4] and parameters: {'reward_function': 1, 'algorithm': 'ddqn', 'n_layers': 2, 'n_units': 512, 'training_rounds': 27, 'learning_rate': 6.64070527729143e-05, 'discount_factor': 0.9561062047837683, 'batch_size': 128, 'target_update_freq': 10, 'soft_update_tau': 0.9516759849416108, 'is_conservative': True, 'lstm': False, 'conservative_alpha': 0.9487301482144148, 'replay_buffer_size': 2851, 'learn_after_episode': False, 'learning_steps': 54, 'n_epochs': 500}.


profit 1150.5957618769667 n_trades 18.4


  0%|          | 0/100 [00:00<?, ?it/s]

...

profit 1183.7035614485405 n_trades 20.0


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [02:37<?, ?it/s]
Profit: 826.0222054107403, Number of Trades: 21.2: 100%|██████████| 100/100 [02:06<00:00,  1.27s/it]             
[I 2024-12-05 04:42:17,568] Trial 99 finished with values: [826.0222054107403, 21.2] and parameters: {'reward_function': 1, 'algorithm': 'ddqn', 'n_layers': 3, 'n_units': 64, 'training_rounds': 9, 'learning_rate': 9.987000521254206e-05, 'discount_factor': 0.9687608589096867, 'batch_size': 64, 'target_update_freq': 5, 'soft_update_tau': 0.4358153156186866, 'is_conservative': False, 'lstm': True, 'conservative_alpha': 0.9609207212374272, 'replay_buffer_size': 1247, 'learn_after_episode': True, 'learning_steps': 24, 'n_epochs': 100}.


profit 826.0222054107403 n_trades 21.2


In [20]:
# print(f"Best value: {study.best_value} (params: {study.best_params})")
best_trials=study.best_trials
best_trials

[FrozenTrial(number=45, state=TrialState.COMPLETE, values=[1191.6558529692093, 20.4], datetime_start=datetime.datetime(2024, 11, 24, 2, 9, 29, 249544), datetime_complete=datetime.datetime(2024, 11, 24, 2, 14, 53, 613837), params={'reward_function': 0, 'algorithm': 'dqn', 'n_layers': 1, 'n_units': 256, 'training_rounds': 21, 'learning_rate': 2.8764743562927777e-05, 'discount_factor': 0.911339518183664, 'batch_size': 64, 'target_update_freq': 5, 'soft_update_tau': 0.30917867193935245, 'is_conservative': False, 'lstm': True, 'conservative_alpha': 0.5255327383263214, 'learn_after_episode': True, 'learning_steps': 51, 'n_epochs': 100}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'reward_function': CategoricalDistribution(choices=(0, 1, 2)), 'algorithm': CategoricalDistribution(choices=('dqn', 'ddqn')), 'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units': CategoricalDistribution(choices=(64, 128, 256, 512)), 'training_rounds': IntDistribution(h

In [21]:
best_trials=study.best_trials
best_trail=best_trials[3]
best_params=best_trail.params
reward_func=reward_functions[best_params.pop('reward_function')]
train_env.reward_func=reward_func
test_env.reward_func=reward_func

In [22]:
algo=best_params.pop('algorithm')

learning_params={'learn_after_episode':best_params.pop('learn_after_episode'),
                    'learning_steps':best_params.pop('learning_steps'),
                    'n_epochs':best_params.pop('n_epochs'),
                    }
best_params['hidden_dims']=make_hidden_dims(n_layers=best_params.pop('n_layers'),n_units=best_params.pop('n_units'))
best_params['lstm']=best_params.pop('lstm')
best_params['action_space_dim']=len(train_env.positions)
best_params['observation_space_dim']=train_env.observation_space.shape[0]
if algo=='dqn':
    agent=pearl_utils.create_dqn_model(**best_params)
elif algo=='ddqn':
    agent=pearl_utils.create_ddqn_model(**best_params)

best_params,learning_params

({'training_rounds': 28,
  'learning_rate': 3.502195725430407e-05,
  'discount_factor': 0.9447058427910774,
  'batch_size': 128,
  'target_update_freq': 24,
  'soft_update_tau': 0.5996751853354005,
  'is_conservative': True,
  'conservative_alpha': 0.6505399516357236,
  'hidden_dims': [128, 128],
  'lstm': True,
  'action_space_dim': 2,
  'observation_space_dim': 24},
 {'learn_after_episode': True, 'learning_steps': 72, 'n_epochs': 100})

In [23]:


agent=pearl_utils.train_pearl_model(agent,train_env,**learning_params)

profit,n_trades=pearl_utils.test_pearl_model(agent,test_env)

agent=pearl_utils.train_pearl_model(agent,test_env,**learning_params)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [05:42<?, ?it/s]


In [24]:
import torch

In [25]:
torch.load

<function torch.serialization.load(f: Union[str, os.PathLike, BinaryIO, IO[bytes]], map_location: Union[Callable[[torch.types.Storage, str], torch.types.Storage], torch.device, str, Dict[str, str], NoneType] = None, pickle_module: Any = None, *, weights_only: Optional[bool] = None, mmap: Optional[bool] = None, **pickle_load_args: Any) -> Any>